# EE 451: Communications Systems
## Lecture 25 - Noise Fundamentals

### Learning Objectives

By the end of this notebook, you will be able to:

1. Analyze thermal noise fundamentals and power spectral density
2. Model Additive White Gaussian Noise (AWGN) in communication systems
3. Calculate noise figure and noise temperature for receivers
4. Compare SNR performance of AM vs. FM systems
5. Simulate noise effects on communication signals
6. Measure SNR using signal processing techniques

---

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, fftfreq, fftshift
from scipy.stats import norm

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Physical constants
k_B = 1.38e-23  # Boltzmann constant (J/K)
T_0 = 290  # Room temperature (K)

print("Libraries imported successfully!")
print(f"Boltzmann constant: k = {k_B} J/K")
print(f"Reference temperature: T₀ = {T_0} K")

## Part 1: Thermal Noise Fundamentals

**Thermal noise** (Johnson-Nyquist noise) is a fundamental limit in communication systems, arising from random motion of electrons in conductors.

### Thermal Noise Power
$$P_n = k T B$$

where:
- $k = 1.38 \times 10^{-23}$ J/K (Boltzmann constant)
- $T$ = Temperature (Kelvin)
- $B$ = Bandwidth (Hz)

### Noise Power Spectral Density
At room temperature (290 K):
$$N_0 = kT = -174 \text{ dBm/Hz}$$

In [ ]:
# Calculate thermal noise power for different bandwidths
bandwidths = np.array([1e3, 10e3, 100e3, 1e6, 10e6, 20e6, 100e6])  # Hz
bandwidth_labels = ['1 kHz', '10 kHz', '100 kHz', '1 MHz', '10 MHz', '20 MHz', '100 MHz']

# Noise power at room temperature
P_n_watts = k_B * T_0 * bandwidths
P_n_dBm = 10 * np.log10(P_n_watts * 1000)  # Convert to dBm

# Noise power spectral density
N0_watts_per_Hz = k_B * T_0
N0_dBm_per_Hz = 10 * np.log10(N0_watts_per_Hz * 1000)

# Create visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart of noise power vs bandwidth
colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(bandwidths)))
axes[0].bar(range(len(bandwidths)), P_n_dBm, color=colors)
axes[0].set_xticks(range(len(bandwidths)))
axes[0].set_xticklabels(bandwidth_labels, rotation=45)
axes[0].set_ylabel('Noise Power (dBm)')
axes[0].set_title('Thermal Noise Power vs. Bandwidth (T = 290 K)')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=-100, color='red', linestyle='--', linewidth=2, label='-100 dBm reference')
axes[0].legend()

# Add values on bars
for i, (bw, pn) in enumerate(zip(bandwidth_labels, P_n_dBm)):
    axes[0].text(i, pn + 1, f'{pn:.1f}', ha='center', fontsize=9)

# Noise power vs bandwidth (continuous)
bw_range = np.logspace(3, 9, 100)  # 1 kHz to 1 GHz
pn_range = k_B * T_0 * bw_range
pn_range_dBm = 10 * np.log10(pn_range * 1000)

axes[1].semilogx(bw_range / 1e6, pn_range_dBm, linewidth=2, color='blue')
axes[1].set_xlabel('Bandwidth (MHz)')
axes[1].set_ylabel('Noise Power (dBm)')
axes[1].set_title('Thermal Noise Power (Continuous)')
axes[1].grid(True, alpha=0.3, which='both')

# Add reference lines for common systems
axes[1].axvline(x=0.02, color='green', linestyle='--', alpha=0.7, label='WiFi (20 MHz)')
axes[1].axvline(x=0.2, color='orange', linestyle='--', alpha=0.7, label='FM Radio (200 kHz)')
axes[1].axvline(x=0.01, color='red', linestyle='--', alpha=0.7, label='AM Radio (10 kHz)')
axes[1].legend()

plt.tight_layout()
plt.show()

print("\nThermal Noise Calculations:")
print("═" * 70)
print(f"Noise PSD at T = {T_0} K: N₀ = {N0_dBm_per_Hz:.1f} dBm/Hz")
print(f"Noise PSD (linear): N₀ = {N0_watts_per_Hz:.3e} W/Hz")
print("\nNoise Power for Common Systems:")
print("─" * 70)
for bw_label, bw, pn in zip(bandwidth_labels, bandwidths, P_n_dBm):
    print(f"{bw_label:<15} BW = {bw/1e6:>8.3f} MHz    P_n = {pn:>7.1f} dBm")
print("═" * 70)

## Part 2: Additive White Gaussian Noise (AWGN)

**AWGN** is the standard noise model for communication systems:
- **Additive**: Noise adds to signal ($r(t) = s(t) + n(t)$)
- **White**: Flat power spectral density across all frequencies
- **Gaussian**: Amplitude follows normal distribution

### Statistical Properties
- Mean: $\mu = 0$
- Variance: $\sigma^2 = N_0/2$ (for complex baseband)
- PDF: $p(n) = \frac{1}{\sqrt{2\pi\sigma^2}} e^{-n^2/(2\sigma^2)}$

In [ ]:
# Generate AWGN samples
num_samples = 100000
sigma = 1.0  # Standard deviation
noise = np.random.normal(0, sigma, num_samples)

# Theoretical Gaussian PDF
x = np.linspace(-5*sigma, 5*sigma, 1000)
pdf_theory = norm.pdf(x, 0, sigma)

# Plot
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Time domain
t_noise = np.arange(1000) / 1000  # First 1000 samples
axes[0, 0].plot(t_noise, noise[:1000], linewidth=0.5, color='blue')
axes[0, 0].set_xlabel('Time (normalized)')
axes[0, 0].set_ylabel('Amplitude')
axes[0, 0].set_title('AWGN - Time Domain')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].axhline(y=0, color='k', linewidth=0.5)

# Histogram vs theoretical PDF
axes[0, 1].hist(noise, bins=100, density=True, alpha=0.7, color='blue', label='Simulated')
axes[0, 1].plot(x, pdf_theory, 'r-', linewidth=2, label='Theoretical Gaussian')
axes[0, 1].set_xlabel('Amplitude')
axes[0, 1].set_ylabel('Probability Density')
axes[0, 1].set_title('AWGN - Amplitude Distribution')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Power spectral density
f_noise, Pxx = signal.welch(noise, fs=1000, nperseg=1024)
axes[1, 0].plot(f_noise, 10*np.log10(Pxx), linewidth=1, color='green')
axes[1, 0].set_xlabel('Frequency (Hz)')
axes[1, 0].set_ylabel('PSD (dB/Hz)')
axes[1, 0].set_title('AWGN - Power Spectral Density ("White" Spectrum)')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].axhline(y=10*np.log10(sigma**2), color='red', linestyle='--', 
                   linewidth=2, label=f'Theoretical: {10*np.log10(sigma**2):.1f} dB/Hz')
axes[1, 0].legend()

# Autocorrelation
lags = np.arange(-100, 101)
autocorr = np.correlate(noise[:1000], noise[:1000], mode='same')[400:601]
autocorr = autocorr / np.max(autocorr)  # Normalize

axes[1, 1].plot(lags, autocorr, linewidth=2, color='purple')
axes[1, 1].set_xlabel('Lag')
axes[1, 1].set_ylabel('Autocorrelation (normalized)')
axes[1, 1].set_title('AWGN - Autocorrelation (Delta-like)')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].axhline(y=0, color='k', linewidth=0.5)
axes[1, 1].axvline(x=0, color='k', linewidth=0.5)

plt.tight_layout()
plt.show()

# Statistics
mean_measured = np.mean(noise)
std_measured = np.std(noise)
variance_measured = np.var(noise)

print("\nAWGN Statistical Properties:")
print("═" * 70)
print(f"{'Property':<20} {'Theoretical':<20} {'Measured':<20}")
print("─" * 70)
print(f"{'Mean':<20} {0:<20.6f} {mean_measured:<20.6f}")
print(f"{'Standard Deviation':<20} {sigma:<20.6f} {std_measured:<20.6f}")
print(f"{'Variance':<20} {sigma**2:<20.6f} {variance_measured:<20.6f}")
print("═" * 70)
print("\nKey AWGN Properties:")
print("  • Zero mean (no DC bias)")
print("  • Flat PSD (white spectrum)")
print("  • Delta-like autocorrelation (uncorrelated samples)")
print("  • Gaussian amplitude distribution")

## Part 3: Signal-to-Noise Ratio (SNR) Calculations

**SNR** is the fundamental metric for signal quality:

$$\text{SNR} = \frac{P_{\text{signal}}}{P_{\text{noise}}} = \frac{S}{N}$$

In dB:
$$\text{SNR}_{\text{dB}} = 10 \log_{10}\left(\frac{S}{N}\right) = S_{\text{dBm}} - N_{\text{dBm}}$$

### Energy per Bit to Noise PSD Ratio
$$\frac{E_b}{N_0} = \frac{\text{SNR} \cdot B}{R_b}$$

where $R_b$ is the bit rate.

In [ ]:
# SNR calculation examples
def calculate_snr(signal_power_dBm, bandwidth_Hz, temperature_K=290):
    """Calculate SNR given signal power and bandwidth"""
    noise_power_watts = k_B * temperature_K * bandwidth_Hz
    noise_power_dBm = 10 * np.log10(noise_power_watts * 1000)
    snr_dB = signal_power_dBm - noise_power_dBm
    return snr_dB, noise_power_dBm

# Example systems
systems = [
    ('WiFi (20 MHz)', -50, 20e6),
    ('FM Radio (200 kHz)', -60, 200e3),
    ('AM Radio (10 kHz)', -70, 10e3),
    ('Cellular LTE (10 MHz)', -80, 10e6),
    ('Satellite (1 MHz)', -110, 1e6)
]

print("\nSNR Calculations for Common Systems:")
print("═" * 80)
print(f"{'System':<25} {'Signal (dBm)':<15} {'Noise (dBm)':<15} {'SNR (dB)':<15}")
print("─" * 80)

snr_results = []
for name, signal_dBm, bw in systems:
    snr_dB, noise_dBm = calculate_snr(signal_dBm, bw)
    snr_results.append(snr_dB)
    print(f"{name:<25} {signal_dBm:<15.1f} {noise_dBm:<15.1f} {snr_dB:<15.1f}")

print("═" * 80)

# Visualize SNR for different systems
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart
system_names = [s[0] for s in systems]
colors = ['green' if snr > 20 else 'orange' if snr > 10 else 'red' for snr in snr_results]

axes[0].barh(range(len(system_names)), snr_results, color=colors)
axes[0].set_yticks(range(len(system_names)))
axes[0].set_yticklabels(system_names)
axes[0].set_xlabel('SNR (dB)')
axes[0].set_title('SNR Comparison Across Systems')
axes[0].grid(True, alpha=0.3, axis='x')
axes[0].axvline(x=10, color='orange', linestyle='--', linewidth=2, alpha=0.5, label='10 dB threshold')
axes[0].axvline(x=20, color='green', linestyle='--', linewidth=2, alpha=0.5, label='20 dB threshold')
axes[0].legend()

# Add values on bars
for i, snr in enumerate(snr_results):
    axes[0].text(snr + 1, i, f'{snr:.1f} dB', va='center', fontsize=10)

# SNR vs Eb/N0 relationship
bit_rates = np.logspace(3, 7, 100)  # 1 kbps to 10 Mbps
bandwidth = 1e6  # 1 MHz
snr_linear = 100  # SNR = 20 dB

EbN0_dB = 10 * np.log10(snr_linear * bandwidth / bit_rates)

axes[1].semilogx(bit_rates / 1e3, EbN0_dB, linewidth=2, color='blue')
axes[1].set_xlabel('Bit Rate (kbps)')
axes[1].set_ylabel('E_b/N_0 (dB)')
axes[1].set_title(f'E_b/N_0 vs Bit Rate (SNR = 20 dB, BW = {bandwidth/1e6} MHz)')
axes[1].grid(True, alpha=0.3, which='both')
axes[1].axhline(y=10, color='red', linestyle='--', linewidth=2, label='10 dB (BER ~ 10⁻⁵ for BPSK)')
axes[1].legend()

plt.tight_layout()
plt.show()

print("\nKey Insights:")
print("  • Higher bandwidth → More noise power → Lower SNR (for same signal)")
print("  • Satellite links have very low SNR (weak signal, long distance)")
print("  • E_b/N_0 decreases as bit rate increases (for fixed SNR and BW)")

## Part 4: Noise in AM Systems

For **Amplitude Modulation (AM)** with sinusoidal message:

$$\text{SNR}_{\text{output}} = \frac{\mu^2}{2} \cdot \text{SNR}_{\text{input}}$$

where $\mu$ is the modulation index.

For $\mu = 1$ (100% modulation):
$$\text{SNR}_{\text{output}} = \frac{1}{2} \cdot \text{SNR}_{\text{input}} \quad \text{(−3 dB degradation)}$$

**DSB-SC** (Double Sideband Suppressed Carrier):
$$\text{SNR}_{\text{output}} = \text{SNR}_{\text{input}} \quad \text{(no degradation)}$$

In [ ]:
# Simulate AM with noise
fs = 50000  # Sampling frequency
t = np.arange(0, 0.1, 1/fs)  # 100 ms

# Message signal (1 kHz sinusoid)
f_m = 1000
m_t = np.sin(2 * np.pi * f_m * t)

# Carrier (10 kHz)
f_c = 10000
c_t = np.cos(2 * np.pi * f_c * t)

# AM modulation with different modulation indices
mu_values = [0.5, 1.0]
snr_input_dB = 30  # Input SNR
snr_input_linear = 10**(snr_input_dB / 10)

fig, axes = plt.subplots(len(mu_values), 2, figsize=(14, 10))

for idx, mu in enumerate(mu_values):
    # AM signal
    s_am = (1 + mu * m_t) * c_t
    
    # Signal power
    P_signal = np.mean(s_am**2)
    
    # Noise power for desired input SNR
    P_noise = P_signal / snr_input_linear
    noise = np.sqrt(P_noise) * np.random.randn(len(t))
    
    # Received signal
    r_t = s_am + noise
    
    # Calculate output SNR (theoretical)
    snr_output_linear = (mu**2 / 2) * snr_input_linear
    snr_output_dB = 10 * np.log10(snr_output_linear)
    degradation_dB = snr_input_dB - snr_output_dB
    
    # Plot clean signal
    plot_samples = int(0.005 * fs)  # Show 5 ms
    axes[idx, 0].plot(t[:plot_samples] * 1000, s_am[:plot_samples], linewidth=1, 
                     color='blue', label='Clean AM')
    axes[idx, 0].plot(t[:plot_samples] * 1000, (1 + mu * m_t[:plot_samples]), '--', 
                     linewidth=2, color='green', alpha=0.7, label='Envelope')
    axes[idx, 0].plot(t[:plot_samples] * 1000, -(1 + mu * m_t[:plot_samples]), '--', 
                     linewidth=2, color='green', alpha=0.7)
    axes[idx, 0].set_xlabel('Time (ms)')
    axes[idx, 0].set_ylabel('Amplitude')
    axes[idx, 0].set_title(f'AM Signal (μ = {mu}, Clean)')
    axes[idx, 0].legend()
    axes[idx, 0].grid(True, alpha=0.3)
    
    # Plot noisy signal
    axes[idx, 1].plot(t[:plot_samples] * 1000, r_t[:plot_samples], linewidth=1, 
                     color='red', alpha=0.7, label='Noisy AM')
    axes[idx, 1].plot(t[:plot_samples] * 1000, (1 + mu * m_t[:plot_samples]), '--', 
                     linewidth=2, color='green', alpha=0.5, label='Envelope')
    axes[idx, 1].set_xlabel('Time (ms)')
    axes[idx, 1].set_ylabel('Amplitude')
    axes[idx, 1].set_title(f'AM + AWGN (μ = {mu}, SNR_in = {snr_input_dB} dB)')
    axes[idx, 1].legend()
    axes[idx, 1].grid(True, alpha=0.3)
    
    print(f"\nAM Performance (μ = {mu}):")
    print(f"  Input SNR:  {snr_input_dB:.1f} dB")
    print(f"  Output SNR: {snr_output_dB:.1f} dB")
    print(f"  Degradation: {degradation_dB:.1f} dB")

plt.tight_layout()
plt.show()

# Comparison plot
mu_range = np.linspace(0.1, 1.0, 20)
snr_in = 30  # dB
snr_out_am = snr_in + 10 * np.log10(mu_range**2 / 2)

plt.figure(figsize=(10, 6))
plt.plot(mu_range, snr_out_am, linewidth=2, color='blue', label='AM')
plt.axhline(y=snr_in, color='green', linestyle='--', linewidth=2, label='DSB-SC / SSB')
plt.xlabel('Modulation Index (μ)')
plt.ylabel('Output SNR (dB)')
plt.title(f'AM Output SNR vs Modulation Index (Input SNR = {snr_in} dB)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.ylim(20, 32)
plt.show()

print("\n" + "═" * 70)
print("AM Noise Performance Summary:")
print("  • AM wastes power in carrier (no information)")
print("  • Maximum efficiency at μ = 1 (100% modulation)")
print("  • Even at μ = 1, output SNR is 3 dB worse than input")
print("  • DSB-SC and SSB have no SNR degradation (no carrier)")
print("═" * 70)

## Part 5: Noise in FM Systems

**Frequency Modulation (FM)** has a unique noise performance:

For sinusoidal modulation:
$$\text{SNR}_{\text{output}} = \frac{3\beta^2(\beta + 1)}{2} \cdot \text{SNR}_{\text{input}}$$

where $\beta$ is the modulation index.

### FM Advantages:
- **Trades bandwidth for SNR**: Higher $\beta$ → wider bandwidth but better SNR
- **FM threshold effect**: Below ~10 dB input SNR, performance degrades rapidly
- **Above threshold**: Significant SNR improvement over AM

In [ ]:
# FM SNR analysis
beta_values = np.array([0.5, 1, 2, 5, 10])
snr_input_range = np.linspace(0, 30, 100)  # Input SNR from 0 to 30 dB
snr_input_linear = 10**(snr_input_range / 10)

# FM threshold
threshold_dB = 10

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# SNR improvement vs beta
for beta in beta_values:
    # FM improvement factor
    improvement_factor = (3 * beta**2 * (beta + 1)) / 2
    snr_output_linear = improvement_factor * snr_input_linear
    
    # Apply threshold effect (simplified model)
    below_threshold = snr_input_range < threshold_dB
    snr_output_linear[below_threshold] *= np.exp(-(threshold_dB - snr_input_range[below_threshold])/3)
    
    snr_output_dB = 10 * np.log10(snr_output_linear)
    
    axes[0].plot(snr_input_range, snr_output_dB, linewidth=2, label=f'β = {beta}')

# AM reference (mu = 1)
snr_output_am = snr_input_range - 3  # -3 dB degradation
axes[0].plot(snr_input_range, snr_output_am, '--', linewidth=2, color='red', label='AM (μ=1)')

axes[0].axvline(x=threshold_dB, color='orange', linestyle='--', linewidth=2, 
               alpha=0.5, label='FM Threshold')
axes[0].set_xlabel('Input SNR (dB)')
axes[0].set_ylabel('Output SNR (dB)')
axes[0].set_title('FM Output SNR vs Input SNR')
axes[0].grid(True, alpha=0.3)
axes[0].legend()
axes[0].set_xlim(0, 30)
axes[0].set_ylim(0, 60)

# FM improvement vs beta (at fixed input SNR)
beta_range = np.linspace(0.5, 15, 100)
snr_input_fixed = 20  # 20 dB input
snr_input_linear_fixed = 10**(snr_input_fixed / 10)

improvement_factor = (3 * beta_range**2 * (beta_range + 1)) / 2
snr_output_fm = 10 * np.log10(improvement_factor * snr_input_linear_fixed)

# Bandwidth vs beta
# Carson's rule: BW ≈ 2(β + 1)f_m
# Normalized to message bandwidth f_m
bw_fm = 2 * (beta_range + 1)

ax2 = axes[1].twinx()

line1 = axes[1].plot(beta_range, snr_output_fm, linewidth=2, color='blue', label='Output SNR')
line2 = ax2.plot(beta_range, bw_fm, linewidth=2, color='green', linestyle='--', label='Bandwidth (normalized)')

axes[1].axhline(y=snr_input_fixed, color='red', linestyle='--', linewidth=2, 
               alpha=0.5, label='Input SNR')

axes[1].set_xlabel('Modulation Index (β)')
axes[1].set_ylabel('Output SNR (dB)', color='blue')
ax2.set_ylabel('Bandwidth (× f_m)', color='green')
axes[1].set_title(f'FM: SNR vs Bandwidth Trade-off (Input SNR = {snr_input_fixed} dB)')
axes[1].grid(True, alpha=0.3)

# Combine legends
lines = line1 + line2 + [axes[1].get_lines()[-1]]
labels = [l.get_label() for l in lines]
axes[1].legend(lines, labels, loc='upper left')

plt.tight_layout()
plt.show()

# Comparison table
print("\nFM vs AM SNR Comparison:")
print("═" * 80)
print(f"{'System':<20} {'Input SNR':<15} {'Output SNR':<15} {'Improvement':<15}")
print("─" * 80)

test_snr = 20  # dB
test_snr_linear = 10**(test_snr / 10)

# AM
am_out = test_snr - 3
print(f"{'AM (μ=1)':<20} {test_snr:<15.1f} {am_out:<15.1f} {am_out - test_snr:<15.1f}")

# FM with different beta
for beta in [1, 2, 5]:
    improvement_factor = (3 * beta**2 * (beta + 1)) / 2
    fm_out = 10 * np.log10(improvement_factor * test_snr_linear)
    improvement = fm_out - test_snr
    print(f"{'FM (β=' + str(beta) + ')':<20} {test_snr:<15.1f} {fm_out:<15.1f} {improvement:<15.1f}")

print("═" * 80)
print("\nKey FM Insights:")
print("  • FM trades bandwidth for SNR improvement")
print("  • Higher β → Better SNR but wider bandwidth")
print("  • FM threshold (~10 dB): Below this, performance degrades rapidly")
print("  • Above threshold: FM significantly outperforms AM")
print("  • Wideband FM (β = 5): 26.5 dB better than AM!")

## Summary and Key Takeaways

### Thermal Noise
- Fundamental limit: $P_n = kTB$
- At room temperature: $N_0 = -174$ dBm/Hz
- Cannot be eliminated, only managed

### AWGN Model
- **A**dditive: $r(t) = s(t) + n(t)$
- **W**hite: Flat power spectral density
- **G**aussian: Normal amplitude distribution
- Standard model for communication system analysis

### SNR Performance
| System | Output SNR | Notes |
|--------|------------|-------|
| AM (μ=1) | SNR_in − 3 dB | Carrier wastes power |
| DSB-SC | SNR_in | No carrier loss |
| SSB | SNR_in | Most efficient AM |
| FM (β=5) | SNR_in + 23.5 dB | Trades BW for SNR |

### Key Design Principles
1. **Minimize bandwidth** when possible (less noise)
2. **Increase signal power** (improves SNR)
3. **Use efficient modulation** (DSB-SC, SSB, FM)
4. **FM advantage**: Above threshold, excellent noise immunity
5. **FM disadvantage**: Below threshold, rapid degradation

### Practical Applications
- **AM radio**: Good coverage, modest noise performance
- **FM radio**: Excellent audio quality (wideband FM, β ≈ 5)
- **Satellite**: Very low SNR, requires powerful coding
- **WiFi/Cellular**: Adaptive modulation based on SNR

### Next Topics
- Matched filtering for optimal SNR
- Bit error rate (BER) vs SNR
- Channel coding for error correction
- MIMO and diversity for fading channels